# Data

Data is [fetched from the Api](api_.py) socket provided, https://services.nvd.nist.gov/rest/json/cves/1.0/. 

A local [MongoDB is established](mongo_.py) and the data is upserted. 

With [mongo queries](pddf.py) the data is unraveled, split into two, for CVSSv2 and CVSSv3. It is then returned as pandas' dataframes.

The `Run()` class will perform all this with default settings

In [3]:

import pandas as pd
import numpy as np

from run import Run

#### Establish and Fill MongoDB
Connects to the API and builds succesive queries by default starting in 2014 waiting 1s between each query.

Upserts to a local MongoDB established with default settings


In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#NOTE: will automatically start downloading and
# try to connect to a default mongoDB client
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Run(collection='t').fill_mongo()


waiting 1s ...
https://services.nvd.nist.gov/rest/json/cves/1.0/?pubStartDate=2014-01-01T00:00:00:000%20UTC-00:00&pubEndDate=2014-05-01T00:00:00:000%20UTC-00:00&resultsPerPage=2000
starting at: 2000 /2205
https://services.nvd.nist.gov/rest/json/cves/1.0/?pubStartDate=2014-01-01T00:00:00:000%20UTC-00:00&pubEndDate=2014-05-01T00:00:00:000%20UTC-00:00&startIndex=2000&resultsPerPage=2000
harvested : 2205 /2205 
0 documents added to  collection,  2205 already exist
waiting 1s ...
https://services.nvd.nist.gov/rest/json/cves/1.0/?pubStartDate=2014-05-01T00:00:00:000%20UTC-00:00&pubEndDate=2014-08-29T00:00:00:000%20UTC-00:00&resultsPerPage=2000
starting at: 2000 /2018
https://services.nvd.nist.gov/rest/json/cves/1.0/?pubStartDate=2014-05-01T00:00:00:000%20UTC-00:00&pubEndDate=2014-08-29T00:00:00:000%20UTC-00:00&startIndex=2000&resultsPerPage=2000
harvested : 2018 /2018 
0 documents added to  collection,  2018 already exist
waiting 1s ...
https://services.nvd.nist.gov/rest/json/cves/1.0/?pubSt

# Part 1
Explore the data to get a better understanding of the content


### Fill Pandas' dataframes
Two df's returned for CVSSv2 and v3 which are merged together using an outer join

- Create a Data frame with one row per CVE id

In [5]:
dfs = Run(collection= 't').fill_df()
dfV2,dfV3 = dfs.dfV2,dfs.dfV3
try:
    df = pd.merge( dfV3, dfV2, 'outer', '_id',suffixes=['_V3', '_V2'])
except:
    print('''ERROR: Possibly no MongoDB loaded\nCreating df from backupDB.csv''')
    pd.read_csv('backupDB.csv', low_memory=False)

- How many CVEs have CVSSv3 metrics versus only CVSSv2 metrics?

In [6]:
print(f"""Total n of CVE's = {len(df)}
with CVSSv3 = {len(df.vectorString_V3.dropna())}
with CVSSv2 = {len(df.vectorString_V2.dropna())}
with just CVSSv2 = {len((df[df['vectorString_V3'].isnull()])['vectorString_V2'].dropna())}
""")

Total n of CVE's = 115262
with CVSSv3 = 100248
with CVSSv2 = 114569
with just CVSSv2 = 14420



Checking for unique values in each column to determine if categorical 

In [7]:
for i in df.columns:
    if df[i].dtype == object and len(df[i].unique()) <10:
        print (df[i].unique(), i)

[nan '3.1' '3.0'] version_V3
[nan 'NETWORK' 'LOCAL' 'PHYSICAL' 'ADJACENT_NETWORK'] attackVector
[nan 'LOW' 'HIGH'] attackComplexity
[nan 'NONE' 'LOW' 'HIGH'] privilegesRequired
[nan 'NONE' 'REQUIRED'] userInteraction
[nan 'UNCHANGED' 'CHANGED'] scope
[nan 'HIGH' 'LOW' 'NONE'] confidentialityImpact_V3
[nan 'HIGH' 'LOW' 'NONE'] integrityImpact_V3
[nan 'HIGH' 'NONE' 'LOW'] availabilityImpact_V3
[nan 'CRITICAL' 'MEDIUM' 'HIGH' 'LOW'] baseSeverity
['2.0' nan] version_V2
['NETWORK' 'LOCAL' 'ADJACENT_NETWORK' nan] accessVector
['LOW' 'MEDIUM' 'HIGH' nan] accessComplexity
['NONE' 'SINGLE' 'MULTIPLE' nan] authentication
['NONE' 'PARTIAL' 'COMPLETE' nan] confidentialityImpact_V2
['PARTIAL' 'NONE' 'COMPLETE' nan] integrityImpact_V2
['NONE' 'PARTIAL' 'COMPLETE' nan] availabilityImpact_V2
['MEDIUM' 'HIGH' 'LOW' nan] severity
[False True nan] obtainAllPrivilege
[False True nan] obtainUserPrivilege
[False True nan] obtainOtherPrivilege
[False True nan] userInteractionRequired
[nan False True] acInsuf

`attackVector` and `accessVector` are nominal categorical and need dummy variables.


In [8]:
Δ_list = [['attackVector','AV_3'], ['accessVector', 'AV_2']]
dum_add = lambda ele: pd.get_dummies(df[ele[0]],prefix=ele[1])

frames = [dum_add(i) for i in Δ_list]

df = df.drop([i[0] for i in Δ_list], axis=1)

frames.append(df)

df = pd.concat(frames,axis=1)


the other columns are ordinal categorical(e.g. `LOW`, `MEDIUM`, `HIGH` ) or boolean and can be filled with a dict

In [9]:
cat_dict = {'NONE': 0, 'LOW':1, 'MEDIUM':2,'HIGH':3, 'CRITICAL':4,
'PARTIAL': 1, 'COMPLETE':2,
'SINGLE' :1, 'MULTIPLE' :2,
'UNCHANGED':0 ,'CHANGED':1,
'REQUIRED':1,
False:0, True:1
}
df.replace(cat_dict, inplace=True)

creating general index of all keys for manipulating columns and adds the new dummy variables

In [13]:
dfV2_keys,dfV3_keys = dfs.dfV2.keys(),dfs.dfV3.keys()


transform_idx = df.T.index

for i in transform_idx:
    #print(i[:4])
    if i[:4] == 'AV_3':
        dfV3_keys = dfV3_keys.append(pd.Index([i]))
    if i[:4] == 'AV_2':
        dfV2_keys = dfV2_keys.append(pd.Index([i]))

dfV3_keys = dfV3_keys.drop('attackVector')
dfV2_keys = dfV2_keys.drop('accessVector')

setDf = dfV3_keys.union(dfV2_keys)#set(dfV3_keys)


creating an index of all columns that appear in both CVSSv2 and v3

In [14]:
idxPairs = []

for i in setDf: 
    substr_i = transform_idx[transform_idx.str.startswith(i+'_')]
    if len(substr_i) >0:
        idxPairs.append(substr_i)
    

#idxPairs

Cleaning up data types

In [15]:
f = df.filter(regex='version*').columns
df[f] = df[f].astype(float)
df = df.convert_dtypes()

- Both CVSSv2 and CVSSv3 have the same set of impact metrics, i.e. Confidentiality, Integrity and Availability, however their values are slightly different. For example, CVSSv2 uses complete (C) to represent the highest level of impact, but CVSSv3 uses high (H) instead. Is it possible to directly map from CVSSv2 impact metric values to CVSSv3?

In [16]:
for i in ['confidentialityImpact','integrityImpact','availabilityImpact']:
    print(df.corr()[i+'_V2'][i+'_V3'])

0.8024715511898207
0.7847626572053179
0.8352650731843352


As the correleation matrix between versions of the suggested metrics is not = 1, it would NOT be a good idea to directly map them

# Part 2
predict the CVSSv3 Scope metric for CVEs without a CVSSv3 vector.

- What type of learning problem is this? What is the target? 

This is a supervised classification problem with a single variable `scope` as the target

In [17]:

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis  import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_absolute_error
from sklearn.preprocessing import StandardScaler

from pprint import pprint


selects and combines the metrics of interest from previous indices

In [18]:

cvssV3 = []
cvssV2 = []
dfV3_keys_cp = dfV3_keys.drop(['_id','vectorString'])
dfV2_keys_cp = dfV2_keys.drop(['_id','vectorString'])
for i in idxPairs:
    if i[0][:-3] in dfV3_keys and i[0][:-3] != 'vectorString':
        cvssV3.append(i[0])
        dfV3_keys_cp = dfV3_keys_cp.drop([i[0][:-3]])

    
    if i[0][:-3] in dfV2_keys and i[0][:-3] != 'vectorString':
        cvssV2.append(i[1])
        dfV2_keys_cp = dfV2_keys_cp.drop([i[0][:-3]])

cvssV3 = dfV3_keys_cp.union(cvssV3)  
cvssV2 = dfV2_keys_cp.union(cvssV2)   


copies the main df, selecting all columns and then dropping types = object (relevant data have dtype float, int,etc), the column `acInsufInfo` and all remaining rows with NaN's 

In [19]:
dfNoNan = df.loc[:,df.dtypes != 'object'].drop('acInsufInfo',axis=1).dropna()

X_pt2 = dfNoNan[cvssV2.drop('acInsufInfo')]
y_pt2 = dfNoNan['scope'].astype(int)


- How would you build the training / validation / testing dataset?

Take a random subset of the data and split into train and test sets

In [20]:
X_pt2_train, X_pt2_test, y_pt2_train, y_pt2_test = train_test_split(X_pt2, y_pt2, test_size= .33) 

intended to scale data however all iterations tried made little difference on this dataset, possibly due to high n of categorical inputs and low variance in numerical data

In [21]:
#sc = StandardScaler()
#X_pt2_train = sc.fit_transform(X_pt2_train)
#X_pt2_test = sc.transform(X_pt2_test)

percentage of total scope count that is = 1



In [22]:
df.scope.value_counts()[1]/df.scope.value_counts()[0]

0.20283647096936755

- Which evaluation metrics would you use?

a helper function which takes in a model, fits it with train values, predicts the test values and checks it against the y test values returning a dict of all results.

NOTE: for compatibility with multioutput models, a custom score() function is defined later on and a value of 0 is given for a multioutput confusion matrix


In [23]:

def model_info(model,X,y):
    #takes model, set of X, set of y
    #returns dict
    X_train, X_test = X
    y_train, y_test = y
    print('~~~ fitting model')
    f = model.fit(X_train.values, y_train.values)
    print('~~~ predicting values')
    ŷ = model.predict(X_test.values)
    print('~~~ checking validity')
    
    try:
        sc = f.score(X_test.values, y_test.values)
    except:
        sc= score(X_test.values, y_test.values)
    
    m = mean_absolute_error(y_test.values, ŷ)
    c = confusion_matrix(y_test.values, ŷ) if len(y_train.shape) == 1 else 0

    dict_ = {'model': f, 'score' : sc, 'prediction': ŷ, 'MAE' : m, 'Confusion Matrix': c}
    return dict_
    

builds a dict of all the model dicts with model name as key and an index corresponding to model list location

In [24]:


def model_dict(mod_list,X,Y):
    #takes a list of models,set of X, set of y
    #returns dict
    mod_dict ={}
    idx = 0
    for i in mod_list: 
        mod_type = i.__str__()
        print(f"\n~~~~~~~~~~~~~~~~~~~~~~~~~\nWorking on {mod_type}")
        if len(Y[0].shape) > 1:
            mod_info = model_info(MultiOutputRegressor(i),X,Y)
        mod_info = model_info(i,X,Y)
        mod_dict[mod_type] =  (idx ,mod_info)
        idx+=1
        pprint(mod_info)
        
    return mod_dict


- What simple model might be used for this problem? Could this be improved upon with a more complex solution?

The Random Forest Classifier is repeatedly the best performer here. Ensemble Learning would be a good canditate here for improving on the given results

In [25]:
model_RFC = RandomForestClassifier()
model_SVC = svm.SVC()
model_MLP = MLPClassifier()
model_LR =  LogisticRegression()
model_LDA =  LinearDiscriminantAnalysis()
model_KNN =  KNeighborsClassifier()
model_CART =  DecisionTreeClassifier()
model_NB =  GaussianNB()

mod_list_pt2 = [
    
    #model_SVC,#slow
    model_MLP,
    model_LR,
    model_LDA,
    model_KNN,
    model_CART,
    model_RFC,
    model_NB
    ]



mod_dict_pt2 = model_dict(mod_list_pt2,[X_pt2_train,X_pt2_test], [y_pt2_train,y_pt2_test])


~~~~~~~~~~~~~~~~~~~~~~~~~
Working on MLPClassifier()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[26633,   592],
       [ 1198,  4334]]),
 'MAE': 0.0546448087431694,
 'model': MLPClassifier(),
 'prediction': array([0, 0, 0, ..., 0, 0, 0]),
 'score': 0.9453551912568307}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on LogisticRegression()
~~~ fitting model


/home/potz/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[26475,   750],
       [ 1529,  4003]]),
 'MAE': 0.0695729157126721,
 'model': LogisticRegression(),
 'prediction': array([0, 0, 0, ..., 0, 0, 0]),
 'score': 0.9304270842873279}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on LinearDiscriminantAnalysis()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[26042,  1183],
       [ 1152,  4380]]),
 'MAE': 0.07128247397502824,
 'model': LinearDiscriminantAnalysis(),
 'prediction': array([0, 0, 0, ..., 0, 0, 0]),
 'score': 0.9287175260249717}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on KNeighborsClassifier()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[26626,   599],
       [ 1232,  4300]]),
 'MAE': 0.055896449613823,
 'model': KNeighborsClassifier(),
 'prediction': array([0, 0, 0, ..., 0, 0, 0]),
 'score': 0.944103550386177}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on DecisionTreeClassifier()
~~~ fitting mo

ŷ dict built from the model dict for easier access to ŷ values

In [49]:
def ŷ_test_dict(mod_dict):
    ŷtest_dict = {}
    for i in mod_dict:
        ŷ = pd.DataFrame(mod_dict[i][1]['prediction'])
        ŷtest_dict[i.__str__()] =  ŷ
    return ŷtest_dict
ŷ_pt2_test_vals = ŷ_test_dict(mod_dict_pt2)

ŷ_pt2_test_vals

{'MLPClassifier()':        0
 0      0
 1      0
 2      0
 3      0
 4      0
 ...   ..
 32752  0
 32753  1
 32754  0
 32755  0
 32756  0
 
 [32757 rows x 1 columns],
 'LogisticRegression()':        0
 0      0
 1      0
 2      0
 3      0
 4      0
 ...   ..
 32752  0
 32753  1
 32754  0
 32755  0
 32756  0
 
 [32757 rows x 1 columns],
 'LinearDiscriminantAnalysis()':        0
 0      0
 1      0
 2      0
 3      0
 4      0
 ...   ..
 32752  0
 32753  1
 32754  0
 32755  0
 32756  0
 
 [32757 rows x 1 columns],
 'KNeighborsClassifier()':        0
 0      0
 1      0
 2      0
 3      0
 4      0
 ...   ..
 32752  0
 32753  1
 32754  0
 32755  0
 32756  0
 
 [32757 rows x 1 columns],
 'DecisionTreeClassifier()':        0
 0      0
 1      0
 2      0
 3      0
 4      0
 ...   ..
 32752  0
 32753  1
 32754  0
 32755  0
 32756  0
 
 [32757 rows x 1 columns],
 'RandomForestClassifier()':        0
 0      0
 1      0
 2      0
 3      0
 4      0
 ...   ..
 32752  0
 32753  1
 32754  

In [28]:
def best_model_chk(mod_dict):
    #takes model dict 
    #returns tuple of index and model score 
    best_mod= ('',0)
    for i in mod_dict:
        presc = mod_dict[i][1]['score']
        sc = np.average(presc) #if isinstance(presc,(int,float)) else np.average(presc)
        if sc > best_mod[1]:
            best_mod = mod_dict[i][0], mod_dict[i][1]['score']

    print(best_mod)
    return best_mod
work_model_pt2 = mod_list_pt2[best_model_chk(mod_dict_pt2)[0]]


(0, 0.9453551912568307)


iterates through all models in the list and produces a dict of results with model name as key 

In [30]:
ŷ_pt2_vals = {}
for i in mod_list_pt2:
    ŷ_pt2 = pd.DataFrame(i.predict(Xnew_pt2.values))
    ŷ_pt2_vals[i.__str__()] = ŷ_pt2.value_counts(), ŷ_pt2

In [31]:
for i in ŷ_pt2_vals:
    print(ŷ_pt2_vals[i][0][1]/ŷ_pt2_vals[i][0][0])

0.16977886977886977
0.16767495094833224
0.18521284540702015
0.16311074918566776
0.17044988937146605
0.17169811320754716
0.5960442507542743


selects data where scope = nan

selects all columns with v2 data except for `acInsufInfo` and drops rows with nan

predicts ŷ based on the best model picked from prev function 

In [84]:
df_y_pt2 = df[df['scope'].isna()]

Xnew_pt2 = df_y_pt2[cvssV2.drop('acInsufInfo')].dropna()

ŷ_pt2 = pd.DataFrame(work_model_pt2.predict(Xnew_pt2.values))

ŷ_pt2.value_counts()

0    12210
1     2073
dtype: int64

predicted `scope` df

In [55]:
ŷ_pt2

,0
0,1
1,1
2,1
3,1
4,1
...,...
14278,1
14279,0
14280,0
14281,0



### Part 3
Predict the CVSSv3 Confidentiality, Integrity and Availability metrics for CVEs without a CVSSv3 vector.
- What type of learning problem is this? What is the target? 


This is a multi output supervised regression problem with 3 targets, `confidentialityImpact_V3`, `integrityImpact_V3`, `availabilityImpact_V3`

each target is calculated as

In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor

model_RFR = RandomForestRegressor()

In [33]:
mod_list_pt3 = [
    
    model_RFR,
    #model_KNN,#slow
    model_CART,
    model_RFC,
    ]
    

In [34]:
dfNoNan = df.loc[:,df.dtypes != 'object'].drop('acInsufInfo',axis=1).dropna()

In [35]:
response = pd.Index(['confidentialityImpact_V3', 'integrityImpact_V3', 'availabilityImpact_V3'])
X = dfNoNan[cvssV2.drop('acInsufInfo')]
y = dfNoNan[response].astype(int)

- How would you build the training / validation / testing dataset?

In [36]:
X_pt3_train, X_pt3_test, y_pt3_train, y_pt3_test = train_test_split(X, y) 


- Which evaluation metrics would you use?

this is a really simple metric based on the sklearn `model.score()` function which gets  1 - total correct divided by total n. My implementation will take multioutput and returns a vector of len(ŷ). Does NOT work for single target ŷ

In [37]:
def score(y_test, ŷ): 
    #takes y known , ŷ
    #returns arr with len(ŷ)

    arr_1 = y_test
    arr_2 = ŷ

    if len(arr_1)!=len(arr_2):
        print(len(arr_1), len(arr_2))
        print('!!! NOT the same length !!!')
        return

    shape = arr_2.shape
 
    truth_d = {True:[0]*shape[1], False:[0]*shape[1]}

    for i in range(shape[0]):
        for j in range(shape[1]):
            truth_d[arr_1[i][j] == arr_2[i][j]][j] += 1

    return [1 - truth_d[True][i]/ shape[0] for i in range(shape[1])]




- What simple model might be used for this problem? Could this be improved upon with a more complex solution?


Here the Multioutput Regressor acts as a wrapper around estimators. This allows for direct regression of each individual estimator. as can be seen from the Random Forest Regressor prediction the output is continous

a better approach would be to use a chained regressor which would chain each regression together in a conditinal manner i.e;   y1  , y2|ŷ1 ,  y3|( ŷ1 & ŷ2 )

In [38]:
mod_dict_pt3 = model_dict(mod_list_pt3,[X_pt3_train,X_pt3_test], [y_pt3_train,y_pt3_test])


~~~~~~~~~~~~~~~~~~~~~~~~~
Working on RandomForestRegressor()
~~~ fitting model
~~~ predicting values
~~~ checking validity
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': 0,
 'MAE': 0.2139516741810863,
 'model': RandomForestRegressor(),
 'prediction': array([[2.87800719, 2.89397179, 2.88352969],
       [0.04612574, 2.09162542, 0.03286848],
       [0.82491744, 1.22633834, 0.01089177],
       ...,
       [0.        , 3.        , 0.        ],
       [0.00967921, 0.00936455, 2.78942462],
       [0.99544568, 1.02547445, 0.00934515]]),
 'score': 0.8720832111398696}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on DecisionTreeClassifier()
~~~ fitting model
~~~ predicting values
~~~ checking validity
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': 0,
 'MAE': 0.13543681495809157,
 'model': DecisionTreeClassifier(),
 'prediction': array([[3, 3, 3],
       [0, 3, 0],
       [1, 1, 0],
       ...,
       [0, 3, 0],
       [0, 0, 3],
     

In [54]:
ŷ_pt3_test_vals = ŷ_test_dict(mod_dict_pt3)


In [71]:
df

,AV_3_ADJACENT_NETWORK,AV_3_LOCAL,AV_3_NETWORK,AV_3_PHYSICAL,AV_2_ADJACENT_NETWORK,AV_2_LOCAL,AV_2_NETWORK,_id,version_V3,vectorString_V3,...,availabilityImpact_V2,baseScore_V2,severity,exploitabilityScore_V2,impactScore_V2,obtainAllPrivilege,obtainUserPrivilege,obtainOtherPrivilege,userInteractionRequired,acInsufInfo
0,0,0,0,0,0,0,1,CVE-2013-5704,<NA>,<NA>,...,0,5.0,2,10.0,2.9,0,0,0,0,<NA>
1,0,0,0,0,0,0,1,CVE-2014-0054,<NA>,<NA>,...,1,6.8,2,8.6,6.4,0,0,0,1,<NA>
2,0,0,0,0,0,0,1,CVE-2013-7315,<NA>,<NA>,...,1,6.8,2,8.6,6.4,0,0,0,0,<NA>
3,0,0,0,0,0,0,1,CVE-2013-4152,<NA>,<NA>,...,1,6.8,2,8.6,6.4,0,0,0,0,<NA>
4,0,0,0,0,0,0,1,CVE-2013-6429,<NA>,<NA>,...,1,6.8,2,8.6,6.4,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115257,0,0,1,0,0,0,1,CVE-2021-45876,3.1,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,...,1,7.5,3,10.0,6.4,0,0,0,0,0
115258,0,0,1,0,0,0,1,CVE-2022-1004,3.1,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:L/I:N/A:N,...,0,4.0,2,8.0,2.9,0,0,0,0,0
115259,0,0,1,0,0,0,1,CVE-2022-24126,3.1,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,...,1,7.5,3,10.0,6.4,0,0,0,0,0
115260,0,0,1,0,0,0,1,CVE-2022-24656,3.1,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:C/C:L/I:L/A:N,...,0,4.3,2,8.6,2.9,0,0,0,1,0


The plan here is as above select all response variables from the df that = NaN

selects all columns with v2 data except for `acInsufInfo` and drop rows with nan

predict ŷ based on the best model picked from above

In [85]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#NOTE: NOT working, having difficulty getting the right pandas indexing 
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
df_y_pt3 = df[df[response ].isna()]
df_y_pt3#.value_counts()

#df[df[['confidentialityImpact_V3', 'integrityImpact_V3', 'availabilityImpact_V3']].isnull()]

#Xnew_pt3 = df_y_pt3[cvssV2.drop('acInsufInfo')].dropna()

#ŷ_pt3 = pd.DataFrame(work_model_pt3.predict(Xnew_pt3.values))


,AV_3_ADJACENT_NETWORK,AV_3_LOCAL,AV_3_NETWORK,AV_3_PHYSICAL,AV_2_ADJACENT_NETWORK,AV_2_LOCAL,AV_2_NETWORK,_id,version_V3,vectorString_V3,...,availabilityImpact_V2,baseScore_V2,severity,exploitabilityScore_V2,impactScore_V2,obtainAllPrivilege,obtainUserPrivilege,obtainOtherPrivilege,userInteractionRequired,acInsufInfo
0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115257,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
115258,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
115259,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
115260,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
